In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/' \
'alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [3]:
df = data.copy()
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


### Data Preparation

In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [20]:
categorical = ['lead_source', 'industry','employment_status', 'location']
numerical = ['interaction_count', 'lead_score','number_of_courses_viewed', 'annual_income']

In [14]:
df[categorical] = df[categorical].fillna('NA')
df[numerical] = df[numerical].fillna(0)

In [16]:
df.isnull().sum()

#all missing values cleared

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### Question 1

In [18]:
df.industry.mode()

#retail is the answer

0    retail
Name: industry, dtype: object

### Question 2

In [21]:
df[numerical].corr()

,interaction_count,lead_score,number_of_courses_viewed,annual_income
interaction_count,1.000000,0.009888,-0.023565,0.027036
lead_score,0.009888,1.000000,-0.004879,0.015610
number_of_courses_viewed,-0.023565,-0.004879,1.000000,0.009770
annual_income,0.027036,0.015610,0.009770,1.000000


- interaction_count and lead_score = 0.009888
- number_of_courses_viewed and lead_score = -0.004879
- number_of_courses_viewed and interaction_count = -0.023565
- annual_income and interaction_count = 0.027036 `Highest`

### Question 3

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [25]:
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [26]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [27]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [29]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [30]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [31]:
df_train_full = df_train_full.reset_index(drop=True)

In [32]:
from sklearn.metrics import mutual_info_score

In [33]:

def mutual_info_churn_score(series):
    return mutual_info_score(series, y_train)


In [35]:
mi = df_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

# Highest is `leadsource`

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

### Question 4

In [36]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [37]:
train_dict[0]

{'lead_source': 'paid_ads',
 'industry': 'retail',
 'employment_status': 'student',
 'location': 'middle_east',
 'interaction_count': 5,
 'lead_score': 0.03,
 'number_of_courses_viewed': 0,
 'annual_income': 58472.0}

In [38]:
from sklearn.feature_extraction import DictVectorizer


In [39]:
dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
# model = LogisticRegression()
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [43]:
model.intercept_[0]

-0.06914728027839123

In [44]:
model.coef_[0].round(3)

array([-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.049,
       -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.311,  0.051,
        0.02 , -0.012, -0.012, -0.115,  0.08 , -0.03 ,  0.004, -0.011,
       -0.011, -0.006,  0.008,  0.006, -0.033, -0.025,  0.454])

In [45]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.61192163, 0.79982617, 0.53021344, 0.47131479, 0.57066132,
       0.44227169, 0.87127669, 0.84883115, 0.83290038, 0.61497802,
       0.54968027, 0.78153088, 0.69039786, 0.77017123, 0.52659441,
       0.91706425, 0.53170635, 0.42123049, 0.30146456, 0.84881583,
       0.79488653, 0.73670375, 0.44527211, 0.64838383, 0.41768821,
       0.75393418, 0.90166116, 0.33903049, 0.43181432, 0.9680681 ,
       0.92018714, 0.37487989, 0.652301  , 0.90650057, 0.75164117,
       0.64202122, 0.82250075, 0.83375553, 0.659116  , 0.30978854,
       0.78942264, 0.35546366, 0.96517759, 0.63389304, 0.51274196,
       0.53230534, 0.82287786, 0.74407401, 0.73452314, 0.68955217,
       0.46964444, 0.84539253, 0.55635244, 0.92637871, 0.65258021,
       0.61526274, 0.63816996, 0.28304018, 0.48049825, 0.57890619,
       0.35497342, 0.62175052, 0.38960779, 0.61156056, 0.85304278,
       0.75430137, 0.89185954, 0.71946459, 0.95387623, 0.89209517,
       0.75277088, 0.3385014 , 0.61376593, 0.51622276, 0.64088

In [46]:
converted_decision = (y_pred >= 0.5)

In [51]:
baseline_accuracy = (y_val == converted_decision).mean()
round(baseline_accuracy, 2)

0.7

The closest to `0.7` is `0.74` 

In [52]:
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')

In [64]:
feature_columns = ['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score']

In [ ]:
name = ['a','b','c']
record = []

for c in name:
    record.append(f'(lat_{c})' )

a
b
c


In [ ]:

for col in feature_columns:
    df_train_ = df_train.drop('lead_source', axis=1)
    df_val_lead_source = df_val.drop('lead_source', axis=1)

In [54]:
df_train_lead_source = df_train.drop('lead_source', axis=1)
df_val_lead_source = df_val.drop('lead_source', axis=1)


In [57]:
dv = DictVectorizer(sparse=False)

train_dict = df_train_lead_source.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val_lead_source.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [58]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [59]:
y_pred = model.predict_proba(X_val)[:, 1]


In [60]:
converted_decision = (y_pred >= 0.5)

In [63]:
current_accuracy = (y_val == converted_decision).mean()
current_accuracy

0.7030716723549488